In [32]:
from typing import List, Tuple
from collections import defaultdict, deque

def find_shortest_paths(N: int, routes: List[Tuple[int, int, int]], start: int) -> Tuple[List[int], List[List[int]]]:
    """
    Find shortest paths using SPFA (Shortest Path Faster Algorithm)
    """
    dist = [float('inf')] * (N + 1)
    dist[start] = 0
    paths = [[] for _ in range(N + 1)]
    paths[start] = [start]
    
    # Create adjacency list
    graph = defaultdict(list)
    for u, v, w in routes:
        graph[u].append((v, w))
    
    # Initialize queue with start node
    queue = deque([start])
    in_queue = [False] * (N + 1)
    in_queue[start] = True
    
    # Count relaxations for each vertex to detect negative cycles
    relaxation_count = [0] * (N + 1)
    
    while queue:
        u = queue.popleft()
        in_queue[u] = False
        
        # Process all neighbors
        for v, w in graph[u]:
            if dist[u] + w < dist[v]:
                dist[v] = dist[u] + w
                paths[v] = paths[u] + [v]
                relaxation_count[v] += 1
                
                # Check for negative cycle
                if relaxation_count[v] >= N:
                    dist[v] = float('-inf')
                    paths[v] = []
                    continue
                
                # Add to queue if not already present
                if not in_queue[v]:
                    queue.append(v)
                    in_queue[v] = True
    
    return dist, paths

def calculate_assignments(N: int, K: int, distances: List[int], paths: List[List[int]], people_needs: List[int], C: int) -> Tuple[List[int], List[Tuple[int, int, int, List[int]]]]:
    """
    Calculate worker assignments to each country
    """
    # Create list of available countries with their distances
    available_countries = []
    for country in range(1, N + 1):
        if people_needs[country - 1] > 0 and distances[country] != float('inf'):
            dist = distances[country]
            available_countries.append((dist, country))
    
    # Sort by total cost (distance + C)
    available_countries.sort(key=lambda x: x[0] if x[0] != float('-inf') else float('-inf'))
    
    results = []
    assignments = []
    workers_assigned = 0
    
    # Assign workers to countries
    for dist, country in available_countries:
        while people_needs[country - 1] > 0 and workers_assigned < K:
            total_cost = dist + C if dist != float('-inf') else float('-inf')
            # Convert -inf to a very negative number for practical purposes
            if total_cost == float('-inf'):
                total_cost = "-inf"
            
            results.append(total_cost)
            assignments.append((workers_assigned + 1, country, total_cost, paths[country]))
            people_needs[country - 1] -= 1
            workers_assigned += 1
    
    # Fill remaining workers with -1
    while workers_assigned < K:
        results.append(-1)
        assignments.append((workers_assigned + 1, None, -1, []))
        workers_assigned += 1
    
    return results, assignments

def runtest(N: int, M: int, C: int, people_needs: List[int], routes: List[Tuple[int, int, int]], K: int):
    """
    Run test with direct data input
    """
    try:
        # Find shortest paths from country 1
        distances, paths = find_shortest_paths(N, routes, 1)
        
        # Calculate assignments
        results, assignments = calculate_assignments(N, K, distances, paths, people_needs.copy(), C)
        
        # Display Results
        print("\nResults:")
        print("-" * 50)
        print(f"{'Worker':<8} {'Country':<10} {'Total Cost':<15} {'Path':<15}")
        print("-" * 50)
        
        for person, country, cost, path in assignments:
            if cost == -1:
                print(f"{person}\t {'N/A'}\t\t {'-1'}\t {'N/A'}")
            else:
                print(f"{person}\t {country}\t\t {cost}\t {path}")
                
        # print("\nCosts:")
        # print(*results, sep='\n')
            
    except Exception as e:
        print(f"Error: {e}")

# Example usage
# runtest(N, M, C, people_needs, routes, K)

In [33]:
# 7.1
N, M, C = 10, 14, 10000
people_needs = [1, 2, 2, 1, 2, 2, 1, 2, 2, 1]
routes = [
    (1, 2, 1),
    (1, 7, 19),
    (1, 9, 13),
    (1, 10, 10),
    (2, 3, 4),
    (3, 4, 14),
    (3, 6, 4),
    (4, 5, 16),
    (4, 7, 6),
    (6, 5, 14),
    (7, 3, 1),
    (8, 4, 15),
    (9, 8, 8),
    (10, 9, 8)
]
K = 20
runtest(N, M, C, people_needs, routes, K)


Results:
--------------------------------------------------
Worker   Country    Total Cost      Path           
--------------------------------------------------
1	 1		 10000	 [1]
2	 2		 10001	 [1, 2]
3	 2		 10001	 [1, 2]
4	 3		 10005	 [1, 2, 3]
5	 3		 10005	 [1, 2, 3]
6	 6		 10009	 [1, 2, 3, 6]
7	 6		 10009	 [1, 2, 3, 6]
8	 10		 10010	 [1, 10]
9	 9		 10013	 [1, 9]
10	 9		 10013	 [1, 9]
11	 4		 10019	 [1, 2, 3, 4]
12	 7		 10019	 [1, 7]
13	 8		 10021	 [1, 9, 8]
14	 8		 10021	 [1, 9, 8]
15	 5		 10023	 [1, 2, 3, 6, 5]
16	 5		 10023	 [1, 2, 3, 6, 5]
17	 N/A		 -1	 N/A
18	 N/A		 -1	 N/A
19	 N/A		 -1	 N/A
20	 N/A		 -1	 N/A


In [34]:
# 7.2

N, M, C = 12, 20, 500
people_needs = [1, 2, 2, 3, 1, 1, 1, 1, 1, 2, 2, 2]
routes = [
    (1, 2, 6),
    (1, 4, 14),
    (1, 12, 11),
    (2, 3, 0),
    (2, 11, 2),
    (3, 8, 2),
    (4, 3, 2),
    (5, 3, 5),
    (5, 7, 4),
    (6, 4, 10),
    (8, 6, 18),
    (8, 7, 12),
    (8, 9, 8),
    (9, 4, 7),
    (9, 10, 20),
    (9, 11, 13),
    (10, 8, 9),
    (11, 5, 9),
    (11, 12, 6),
    (12, 6, 16)
]
K = 20
runtest(N, M, C, people_needs, routes, K)


Results:
--------------------------------------------------
Worker   Country    Total Cost      Path           
--------------------------------------------------
1	 1		 500	 [1]
2	 2		 506	 [1, 2]
3	 2		 506	 [1, 2]
4	 3		 506	 [1, 2, 3]
5	 3		 506	 [1, 2, 3]
6	 8		 508	 [1, 2, 3, 8]
7	 11		 508	 [1, 2, 11]
8	 11		 508	 [1, 2, 11]
9	 12		 511	 [1, 12]
10	 12		 511	 [1, 12]
11	 4		 514	 [1, 4]
12	 4		 514	 [1, 4]
13	 4		 514	 [1, 4]
14	 9		 516	 [1, 2, 3, 8, 9]
15	 5		 517	 [1, 2, 11, 5]
16	 7		 520	 [1, 2, 3, 8, 7]
17	 6		 526	 [1, 2, 3, 8, 6]
18	 10		 536	 [1, 2, 3, 8, 9, 10]
19	 10		 536	 [1, 2, 3, 8, 9, 10]
20	 N/A		 -1	 N/A


In [35]:
# 7.3
N, M, C = 14, 27, 1000
people_needs = [1, 2, 1, 2, 2, 1, 2, 2, 1, 1, 1, 1, 2, 1]
routes = [
    (1, 2, 20),
    (1, 6, 200),
    (1, 10, 400),
    (1, 14, 12),
    (2, 11, 13),
    (3, 12, 2),
    (4, 1, 17),
    (4, 3, 12),
    (4, 7, 15),
    (5, 4, 9),
    (6, 2, 20),
    (6, 4, 2),
    (7, 2, 1),
    (7, 13, 5),
    (8, 3, 16),
    (8, 12, 3),
    (9, 8, 14),
    (9, 13, 19),
    (10, 5, 9),
    (10, 6, 10),
    (11, 5, 15),
    (11, 9, 14),
    (11, 14, 16),
    (12, 3, 18),
    (12, 8, 6),
    (12, 11, 19),
    (13, 4, 16)
]
K = 22
runtest(N, M, C, people_needs, routes, K)


Results:
--------------------------------------------------
Worker   Country    Total Cost      Path           
--------------------------------------------------
1	 1		 1000	 [1]
2	 14		 1012	 [1, 14]
3	 2		 1020	 [1, 2]
4	 2		 1020	 [1, 2]
5	 11		 1033	 [1, 2, 11]
6	 9		 1047	 [1, 2, 11, 9]
7	 5		 1048	 [1, 2, 11, 5]
8	 5		 1048	 [1, 2, 11, 5]
9	 4		 1057	 [1, 2, 11, 5, 4]
10	 4		 1057	 [1, 2, 11, 5, 4]
11	 8		 1061	 [1, 2, 11, 9, 8]
12	 8		 1061	 [1, 2, 11, 9, 8]
13	 12		 1064	 [1, 2, 11, 9, 8, 12]
14	 13		 1066	 [1, 2, 11, 9, 13]
15	 13		 1066	 [1, 2, 11, 9, 13]
16	 3		 1069	 [1, 2, 11, 5, 4, 3]
17	 7		 1072	 [1, 2, 11, 5, 4, 7]
18	 7		 1072	 [1, 2, 11, 5, 4, 7]
19	 6		 1200	 [1, 6]
20	 10		 1400	 [1, 10]
21	 N/A		 -1	 N/A
22	 N/A		 -1	 N/A


In [36]:
# 7.4
N, M, C = 20, 41, 1000
people_needs = [1, 2, 1, 2, 2, 2, 1, 1, 2, 1, 2, 2, 1, 1, 1, 2, 2, 2, 1, 1]

routes = [
    (1, 7, 9),
    (1, 4, 9),
    (1, 10, 12),
    (2, 9, 5),
    (2, 10, 8),
    (2, 11, 7),
    (2, 20, 5),
    (3, 2, 11),
    (3, 12, 9),
    (3, 19, 10),
    (4, 2, 5),
    (5, 6, 10),
    (5, 8, 8),
    (5, 8, 15),
    (6, 2, 10),
    (6, 4, 13),
    (6, 7, 10),
    (6, 8, 12),
    (6, 19, 10),
    (7, 3, 6),
    (7, 10, 19),
    (7, 14, 5),
    (8, 17, 8),
    (9, 6, 5),
    (9, 13, 8),
    (10, 3, 19),
    (10, 5, 2),
    (10, 8, 7),
    (12, 15, 9),
    (13, 11, 12),
    (13, 17, 1),
    (14, 20, 11),
    (15, 13, 11),
    (15, 16, 5),
    (16, 18, 10),
    (17, 5, 12),
    (17, 5, 17),
    (18, 1, 11),
    (18, 14, 11),
    (18, 20, 18),
    (19, 14, 19)
]
K = 40
runtest(N, M, C, people_needs, routes, K)


Results:
--------------------------------------------------
Worker   Country    Total Cost      Path           
--------------------------------------------------
1	 1		 1000	 [1]
2	 4		 1009	 [1, 4]
3	 4		 1009	 [1, 4]
4	 7		 1009	 [1, 7]
5	 10		 1012	 [1, 10]
6	 2		 1014	 [1, 4, 2]
7	 2		 1014	 [1, 4, 2]
8	 5		 1014	 [1, 10, 5]
9	 5		 1014	 [1, 10, 5]
10	 14		 1014	 [1, 7, 14]
11	 3		 1015	 [1, 7, 3]
12	 8		 1019	 [1, 10, 8]
13	 9		 1019	 [1, 4, 2, 9]
14	 9		 1019	 [1, 4, 2, 9]
15	 20		 1019	 [1, 4, 2, 20]
16	 11		 1021	 [1, 4, 2, 11]
17	 11		 1021	 [1, 4, 2, 11]
18	 6		 1024	 [1, 10, 5, 6]
19	 6		 1024	 [1, 10, 5, 6]
20	 12		 1024	 [1, 7, 3, 12]
21	 12		 1024	 [1, 7, 3, 12]
22	 19		 1025	 [1, 7, 3, 19]
23	 13		 1027	 [1, 4, 2, 9, 13]
24	 17		 1027	 [1, 10, 8, 17]
25	 17		 1027	 [1, 10, 8, 17]
26	 15		 1033	 [1, 7, 3, 12, 15]
27	 16		 1038	 [1, 7, 3, 12, 15, 16]
28	 16		 1038	 [1, 7, 3, 12, 15, 16]
29	 18		 1048	 [1, 7, 3, 12, 15, 16, 18]
30	 18		 1048	 [1, 7, 3, 12, 15, 16, 18]
31	

In [37]:
# 7.5
N, M, C = 9, 11, 100
people_needs = [1, 2, 2, 1, 1, 2, 1, 2, 1]

routes = [
    (1, 2, 8),
    (1, 3, 5),
    (1, 5, 6),
    (1, 6, -6),
    (2, 5, 3),
    (3, 4, 2),
    (3, 7, 11),
    (4, 2, -2),
    (7, 8, 3),
    (8, 9, 4),
    (9, 3, -1)
]
K = 15
runtest(N, M, C, people_needs, routes, K)


Results:
--------------------------------------------------
Worker   Country    Total Cost      Path           
--------------------------------------------------
1	 6		 94	 [1, 6]
2	 6		 94	 [1, 6]
3	 1		 100	 [1]
4	 2		 105	 [1, 3, 4, 2]
5	 2		 105	 [1, 3, 4, 2]
6	 3		 105	 [1, 3]
7	 3		 105	 [1, 3]
8	 5		 106	 [1, 5]
9	 4		 107	 [1, 3, 4]
10	 7		 116	 [1, 3, 7]
11	 8		 119	 [1, 3, 7, 8]
12	 8		 119	 [1, 3, 7, 8]
13	 9		 123	 [1, 3, 7, 8, 9]
14	 N/A		 -1	 N/A
15	 N/A		 -1	 N/A


In [38]:
# 7.6
N, M, C = 8, 11, 100
people_needs = [1, 2, 1, 2, 2, 2, 1, 1]

routes = [
    (1, 2, 8),
    (1, 6, 4),
    (2, 3, 3),
    (2, 5, 5),
    (2, 8, 20),
    (4, 3, -10),
    (5, 4, 2),
    (5, 7, 12),
    (6, 7, -4),
    (8, 1, 6),
    (8, 3, 1)
]
K = 15
runtest(N, M, C, people_needs, routes, K)


Results:
--------------------------------------------------
Worker   Country    Total Cost      Path           
--------------------------------------------------
1	 1		 100	 [1]
2	 7		 100	 [1, 6, 7]
3	 6		 104	 [1, 6]
4	 6		 104	 [1, 6]
5	 3		 105	 [1, 2, 5, 4, 3]
6	 2		 108	 [1, 2]
7	 2		 108	 [1, 2]
8	 5		 113	 [1, 2, 5]
9	 5		 113	 [1, 2, 5]
10	 4		 115	 [1, 2, 5, 4]
11	 4		 115	 [1, 2, 5, 4]
12	 8		 128	 [1, 2, 8]
13	 N/A		 -1	 N/A
14	 N/A		 -1	 N/A
15	 N/A		 -1	 N/A


In [39]:
# 7.7
N, M, C = 11, 17, 100
people_needs = [1, 1, 2, 2, 1, 1, 2, 2, 1, 1, 2]
routes = [
    [1, 2, -1],
    [1, 6, 4],
    [1, 11, 2],
    [2, 3, -2],
    [2, 4, 9],
    [3, 4, -1],
    [3, 7, 1],
    [4, 5, -2],
    [5, 6, -1],
    [6, 7, -2],
    [7, 8, -1],
    [8, 1, -2],
    [8, 3, 5],
    [8, 5, 10],
    [9, 10, -2],
    [10, 11, -1],
    [11, 9, -1],
]
K = 20

# Run the test
runtest(N, M, C, people_needs, routes, K)


Results:
--------------------------------------------------
Worker   Country    Total Cost      Path           
--------------------------------------------------
1	 6		 -inf	 []
2	 7		 -inf	 []
3	 7		 -inf	 []
4	 11		 -inf	 []
5	 11		 -inf	 []
6	 5		 48	 [1, 2, 3, 4, 5, 6, 7, 8, 1, 2, 3, 7, 8, 1, 2, 3, 4, 5, 6, 7, 8, 1, 2, 3, 4, 5, 6, 7, 8, 1, 2, 3, 7, 8, 1, 2, 3, 4, 5]
7	 4		 50	 [1, 2, 3, 4, 5, 6, 7, 8, 1, 2, 3, 7, 8, 1, 2, 3, 4, 5, 6, 7, 8, 1, 2, 3, 4, 5, 6, 7, 8, 1, 2, 3, 7, 8, 1, 2, 3, 4]
8	 4		 50	 [1, 2, 3, 4, 5, 6, 7, 8, 1, 2, 3, 7, 8, 1, 2, 3, 4, 5, 6, 7, 8, 1, 2, 3, 4, 5, 6, 7, 8, 1, 2, 3, 7, 8, 1, 2, 3, 4]
9	 3		 51	 [1, 2, 3, 4, 5, 6, 7, 8, 1, 2, 3, 7, 8, 1, 2, 3, 4, 5, 6, 7, 8, 1, 2, 3, 4, 5, 6, 7, 8, 1, 2, 3, 7, 8, 1, 2, 3]
10	 3		 51	 [1, 2, 3, 4, 5, 6, 7, 8, 1, 2, 3, 7, 8, 1, 2, 3, 4, 5, 6, 7, 8, 1, 2, 3, 4, 5, 6, 7, 8, 1, 2, 3, 7, 8, 1, 2, 3]
11	 2		 53	 [1, 2, 3, 4, 5, 6, 7, 8, 1, 2, 3, 7, 8, 1, 2, 3, 4, 5, 6, 7, 8, 1, 2, 3, 4, 5, 6, 7, 8, 1, 2, 3, 7, 8, 1, 2]
12	 1